In [39]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
import openai
import io
import pandas as pd
import tqdm
import sys
from datetime import datetime


sys.path.append('../hot_links')

from hot_links.vector_db_utils import *


## OpenAI Vector Store

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client = OpenAI()


In [16]:
def create_vs(name):
    vector_store = client.vector_stores.create(
        name=name
    )
    return vector_store.id

# Create the vector store

In [17]:
def make_file_object(name,content):
    return (name+'.txt', io.BytesIO(content.encode('utf-8')), 'text/markdown')
# Important to encode as bytes and add '.txt' to filename to trick API that this is a file

In [24]:
def push_file_to_cloud(name, content):
    result = client.files.create(file = make_file_object(name, content), purpose = 'assistants') 
    return result.id

In [21]:
def add_file_to_db(file_id, vs_id, attributes = {}):
    result = client.vector_stores.files.create(
        vector_store_id=vs_id,
        file_id=file_id,
        attributes=attributes
    )
    return result
# Then add it to the vector store with attributes

In [3]:
now = datetime.now()
unix_time = now.timestamp()
# Make a time attribute


In [4]:
unix_time 

1756282681.083156

In [5]:
vector_store_id = create_vs("test_12")
file_id = push_file_to_cloud("test_12", "The key to the universe is 24")
res = add_file_to_db(file_id, vector_store_id, attributes={'time':unix_time})

In [6]:
res

VectorStoreFile(id='file-YAwezHhAVEmax1MrFbFr7E', created_at=1756282693, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_68aebf4484b48191981bd5f0c89a35ca', attributes={'time': 1756282681.083156}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [7]:
response = client.responses.create(
    model="gpt-4.1",
    input="I need the key to the universe... What might it be? Please check in the vector store.",
    tools=[{
        "type": "file_search",
        "vector_store_ids": [vector_store_id],
        "filters": {
            "type": "lt",
            "key": "time",
            "value": datetime.now().timestamp()
        }
    }]
)
print(response)
# Then query the store with filters

Response(id='resp_68aebf55f1d881908ae21ab8523f05de06121715ae4800a9', created_at=1756282710.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68aebf56eff88190b6e9234825c1b68206121715ae4800a9', queries=['key to the universe'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68aebf5801e48190be3e51b48fcfa36306121715ae4800a9', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-YAwezHhAVEmax1MrFbFr7E', filename='test_12.txt', index=29, type='file_citation')], text='The key to the universe is 24.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68aebf4484b48191981bd5f0c89a35ca'], filters=ComparisonFilter(key='time', type='lt', value=1756282708.7698

In [8]:
response

Response(id='resp_68aebf55f1d881908ae21ab8523f05de06121715ae4800a9', created_at=1756282710.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFileSearchToolCall(id='fs_68aebf56eff88190b6e9234825c1b68206121715ae4800a9', queries=['key to the universe'], status='completed', type='file_search_call', results=None), ResponseOutputMessage(id='msg_68aebf5801e48190be3e51b48fcfa36306121715ae4800a9', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-YAwezHhAVEmax1MrFbFr7E', filename='test_12.txt', index=29, type='file_citation')], text='The key to the universe is 24.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FileSearchTool(type='file_search', vector_store_ids=['vs_68aebf4484b48191981bd5f0c89a35ca'], filters=ComparisonFilter(key='time', type='lt', value=1756282708.7698

In [9]:
#response.error
response.output[1].content[0].text

'The key to the universe is 24.'

In [11]:
for vs in get_all_vs():
    print(vs.id,vs.name)

vs_68aebf4484b48191981bd5f0c89a35ca test_12
vs_68aebb80bbb08191ac68f228a08a83d4 test_11
vs_68aebae352f48191927c4b1260fb7f73 test_10
vs_68aeba2add0c81919da4e4d10fc47630 test_9
vs_68aeb9afea2081919f8be4f1ee611a82 test_8
vs_68aeb89b6c088191b75c037f1405d714 test_7
vs_68aeb734ba888191830f01f1ec997b47 test_6
vs_68aeb6ef7fb88191b1698338b66a6eec test_5
vs_68aeb6bfb31c81919e722bbd2f50c7b5 test_4
vs_68acb4af008c81919b4924472e209446 hot_links
vs_68acae441e188191a605f82b03a98305 hot_links
vs_68ac553757d88191873c6900f9c90c0c hot_links_final2
vs_68ac54fb91b0819180b65563cb88062e hot_links_final2
vs_68ac547571e881918c5b003281473561 hot_links_final
vs_68ac4cff5f1c81919b3bfda45ea803bb hot_links
vs_68ab6e552c348191b066ae9baa26b73f hot_links
vs_68ab680fb1908191b4c9cf36e59ab145 hot_links
vs_68ab595999388191ad48c47f6d09bae2 hot_links


In [14]:
delete_all_files()

10it [00:04,  2.06it/s]


## Try adding real files

In [17]:
df = pd.read_csv('../data/interim/snowballed.csv')

In [20]:
df.head()

,Unnamed: 0,article_id,sectionName,webTitle,webUrl,bodyContent,webPublicationDate,id
0,147918,environment/2022/jun/30/former-australian-chie...,Environment,Former Australian chief scientist to head revi...,https://www.theguardian.com/environment/2022/j...,The former Australian chief scientist and seni...,2022-06-30T23:53:09Z,147919
1,147919,world/2022/jul/01/we-were-too-lenient-on-pro-d...,World news,We were too lenient on pro-democracy politicia...,https://www.theguardian.com/world/2022/jul/01/...,A senior member of Hong Kong’s incoming admini...,2022-06-30T23:52:37Z,147920
2,147920,australia-news/2022/jul/01/south-australian-li...,Australia news,South Australian Liberal leader and state MPs ...,https://www.theguardian.com/australia-news/202...,A week after Roe v Wade was overturned in the ...,2022-06-30T23:33:09Z,147921
3,147921,australia-news/2022/jul/01/john-barilaros-deci...,Australia news,John Barilaro’s decision may stem some bleedin...,https://www.theguardian.com/australia-news/202...,John Barilaro didn’t want to continue being “a...,2022-06-30T23:23:38Z,147922
4,147922,lifeandstyle/2022/jul/01/breastfeeding-cogniti...,Life and style,Breastfeeding improves cognitive ability for c...,https://www.theguardian.com/lifeandstyle/2022/...,Children of poorer mothers who breastfeed are ...,2022-06-30T23:01:03Z,147923


In [48]:
vector_store_id = create_vs("test_14")


In [50]:
for row in tqdm.tqdm(df.head().iterrows()):
    date,url,content = row[1][['webPublicationDate', 'webUrl','bodyContent']]

    file_id = push_file_to_cloud(url, content)
    res = add_file_to_db(file_id, vector_store_id, attributes={'time':convert_date_to_epoch(date)})


0it [00:00, ?it/s]

5it [00:06,  1.23s/it]


In [54]:
pd.to_datetime('2023-01-01').value//10**9

1672531200

In [61]:
response = query_db(vector_store_id, "What are the positive effects of breastfeeding for poor mothers?", pd.to_datetime('2023-01-01').value//10**9)
print(response.output[1].content[0].text)

I did not find specific information about the positive effects of breastfeeding for poor mothers in your uploaded files. However, based on well-established research, here are some key positive effects of breastfeeding for poor mothers:

1. **Economic Savings**: Breastfeeding eliminates or reduces the need to buy infant formula, bottles, and sterilizing equipment, which can be a significant cost saving for families with limited financial resources.

2. **Improved Maternal Health**: Breastfeeding is linked to faster postpartum recovery, a lower risk of certain cancers (such as breast and ovarian), reduced risk of type 2 diabetes, and decreased risk of postpartum depression, supporting overall well-being in mothers who may have limited access to healthcare.

3. **Enhanced Child Health**: Breastfed infants are less likely to suffer from infections, diarrhea, and respiratory illnesses, which can reduce healthcare expenses and the physical/emotional burden on mothers.

4. **Bonding and Emoti

In [62]:
response = query_db(vector_store_id, "What are the positive effects of breastfeeding for poor mothers?", pd.to_datetime('2020-01-01').value//10**9)
print(response.output[1].content[0].text)

Breastfeeding has several positive effects for poor mothers, particularly regarding the development and future success of their children:

1. **Better Cognitive Development for Children:** Studies show that children of poorer mothers who breastfeed do significantly better on tasks involving speaking, drawing, and comprehension. For example, breastfed children of disadvantaged mothers perform 8% better in cognitive ability tests up to age seven than those who are bottle-fed. At age three, these children score nearly 10 points higher in expressive language and around 8 points higher in school readiness than the average. These cognitive benefits persist at ages five and seven, including visuo-spatial skills and pictorial reasoning.

2. **Preparation for School:** The improved cognitive skills gained from breastfeeding prepare children better for starting primary school, which can be especially valuable for children from disadvantaged backgrounds.

3. **Cost Savings:** For poor mothers, br

In [ ]:
print(response.output[0])
# The query

ResponseFileSearchToolCall(id='fs_68aec71b15f08194a58884a881e09e6206e469958242e74e', queries=['positive effects of breastfeeding for poor mothers', 'benefits of breastfeeding among low-income women', 'advantages of breastfeeding in poverty', 'breastfeeding impact on poor mothers', 'maternal health benefits breastfeeding low income'], status='completed', type='file_search_call', results=None)


In [ ]:
print(response.output[1])
# The output

ResponseOutputMessage(id='msg_68aec71dae388194bd0864fe008cfa8a06e469958242e74e', content=[ResponseOutputText(annotations=[AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=737, type='file_citation'), AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=956, type='file_citation'), AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=1618, type='file_citation')], text='Breastfeeding has several positive effects for poor mothers, particularly regarding the development and future success of their children:\n\n1. **Better Cognitive Development for C

In [73]:
for a in response.output[1].content[0].annotations:
    print(a)
# The output

AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=737, type='file_citation')
AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=956, type='file_citation')
AnnotationFileCitation(file_id='file-GqmYmtuhqxxaFo9UR1fyrU', filename='https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt', index=1618, type='file_citation')


In [83]:
def search_db(vector_store_id, query):
    results = client.vector_stores.search(
        vector_store_id=vector_store_id,
        query=query,
    )
    return results

In [84]:
search_db(vector_store_id, "What are the effects of breastfeeding?")

SyncPage[VectorStoreSearchResponse](data=[VectorStoreSearchResponse(attributes={'time': 1656630063.0}, content=[Content(text='Children of poorer mothers who breastfeed are much better at tasks involving speaking, drawing and comprehension as a direct result, a British study has found. They do 8% better in tests of their cognitive ability up to the age of seven – if they were breastfed for at least three months – than those who were bottle-fed, according to the research. Breastfeeding boosts brain development among the offspring of disadvantaged mothers to such an extent that they are better prepared than children reared on formula milk to start primary school. The findings provide significant new evidence that breastfeeding is good for children. Nevertheless, while previous studies have mainly linked intake of breast milk at an early age to physical health, this new paper – based on analysis of data on almost 6,000 British children – is one of the first to show that it improves their c

In [85]:
for r in results:
    print(r.score)
    print(r.filename)
    print(r.content)
    print()

0.8933144160281347
https://www.theguardian.com/lifeandstyle/2022/jul/01/breastfeeding-cognitive-ability-children-of-poorer-mothers-study.txt
[Content(text='Children of poorer mothers who breastfeed are much better at tasks involving speaking, drawing and comprehension as a direct result, a British study has found. They do 8% better in tests of their cognitive ability up to the age of seven – if they were breastfed for at least three months – than those who were bottle-fed, according to the research. Breastfeeding boosts brain development among the offspring of disadvantaged mothers to such an extent that they are better prepared than children reared on formula milk to start primary school. The findings provide significant new evidence that breastfeeding is good for children. Nevertheless, while previous studies have mainly linked intake of breast milk at an early age to physical health, this new paper – based on analysis of data on almost 6,000 British children – is one of the first to

Based on [OpenAI documentation](https://platform.openai.com/docs/guides/tools-file-search#how-to-use)

In [14]:
b = io.BytesIO(b"some initial text data")

In [16]:
b.getvalue()

b'some initial text data'

In [22]:
from openai.types import FileObject

In [24]:
?FileObject

Init signature:
FileObject(
    *,
    id: str,
    bytes: int,
    created_at: int,
    filename: str,
    object: Literal['file'],
    purpose: Literal['assistants', 'assistants_output', 'batch', 'batch_output', 'fine-tune', 'fine-tune-results', 'vision', 'user_data'],
    status: Literal['uploaded', 'processed', 'error'],
    expires_at: Optional[int] = None,
    status_details: Optional[str] = None,
    **extra_data: Any,
) -> None
Docstring:     
!!! abstract "Usage Documentation"
    [Models](../concepts/models.md)

A base class for creating Pydantic models.

Attributes:
    __class_vars__: The names of the class variables defined on the model.
    __private_attributes__: Metadata about the private attributes of the model.
    __signature__: The synthesized `__init__` [`Signature`][inspect.Signature] of the model.

    __pydantic_complete__: Whether model building is completed, or if there are still undefined fields.
    __pydantic_core_schema__: The core schema of the model.
   

In [45]:
io.BytesIO('test'.encode('utf-8'))

In [14]:
make_file_object('dummy_3.txt', "Wine is made from grapes")

('dummy_3.txt.txt', <_io.BytesIO at 0x76e349562ca0>, 'text/markdown')

In [28]:
result.id

'file-RJNaMAB5ipLTiv696Tai1e'

In [6]:
def add_file_object(file_object):
    client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=file_object   
)

In [7]:
client.vector_stores.files.upload_and_poll(        # Upload file
    vector_store_id=vector_store.id,
    file=make_file_object('dummy_3.txt', "Wine is made from grapes")   
)

VectorStoreFile(id='file-Vqdvu9xspCKE49JZ6WoYFr', created_at=1756148927, last_error=None, object='vector_store.file', status='completed', usage_bytes=1048, vector_store_id='vs_68acb4af008c81919b4924472e209446', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))

In [8]:
client.files.retrieve("file-Vqdvu9xspCKE49JZ6WoYFr")

FileObject(id='file-Vqdvu9xspCKE49JZ6WoYFr', bytes=24, created_at=1756148925, filename='dummy_3.txt.txt', object='file', purpose='assistants', status='processed', expires_at=None, status_details=None)

In [9]:
results = client.vector_stores.search(
    vector_store_id=vector_store.id,
    query="How is wine made?",
)

In [10]:
results

SyncPage[VectorStoreSearchResponse](data=[VectorStoreSearchResponse(attributes={}, content=[Content(text='Wine is made from grapes', type='text')], file_id='file-Vqdvu9xspCKE49JZ6WoYFr', filename='dummy_3.txt.txt', score=0.5208766983986627)], object='vector_store.search_results.page', search_query=['How is wine made?'], has_more=False, next_page=None)

In [65]:
??make_file_object

Signature: make_file_object(name, content)
Docstring: <no docstring>
Source:   
def make_file_object(name,content):
    return (name, io.BytesIO(content.encode('utf-8')), 'text/plain')
File:      /tmp/ipykernel_709/3135118408.py
Type:      function

In [11]:
add_file_object(make_file_object('xxx','lorem ipsum'))

In [ ]:
add_file_object(make_file_object('xxx',df.head()['bodyContent'].values[0]))

BadRequestError: Error code: 400 - {'error': {'message': 'File type not supported', 'type': 'invalid_request_error', 'param': None, 'code': None}}

In [85]:
df.head(10).apply(lambda row: make_file_object(row['webUrl'], row['bodyContent']), axis=1).apply(add_file_object)